# Performance by product --> SoM Dashboard

In [1]:
BASE_DIR = '/Users/efraflores/Desktop/EF/Corner/Brands/SoM/data/'
CPG = 'rb_hyho' 
nielsen_cat = False #True JUST FOR RB_HYHO, DANONE, UNILEVER SO FAR!!!
past = True
SOM_CPG = CPG.title()

## Import

In [2]:
import os
import numpy as np
import pandas as pd
from IPython.display import display

pd.options.display.float_format = '{:.2f}'.format
print([x for x in os.listdir(BASE_DIR) if x.endswith(f'{CPG}.csv')])

['nielsen_rb_hyho.csv', 'products_rb_hyho.csv', 'som_rb_hyho.csv']


In [3]:
df = pd.read_csv(os.path.join(BASE_DIR,f'products_{CPG}.csv'),
                 sep='\t', encoding='UTF-16', low_memory=False)
df.sample(2)

,month,country,city name,store_name,category en,category_id,product id,product name,barcodes,package,...,sales,orders,avg ticket,users,units found,units replaced,units requested,found rate,fulfillment,frequency
28674,septiembre 2021,MX,Monterrey,HEB,Oral care,967,178292,Pasta dental Max White complete clean,{7460192001743},2 x 75 ml,...,0,0,NaN,0,0,2,2,0.0%,100.0%,NaN
22185,septiembre 2021,MX,León,HEB,Cleaning accessories,987,1986567,Guantes baño y lavandería grandes,{51135814215},1 par,...,57,1,57,1,1,0,1,100.0%,100.0%,1.00


In [4]:
# df.rename(columns={'month  / year':'month  ','sales [currency]':'sales'}, inplace=True)

In [5]:
aux_ini = df.copy()
aux_ini['sales'] = aux_ini['sales'].astype(str).str.replace(',','').astype(float)
aux_ini = aux_ini.groupby('month  ')['sales'].sum().reset_index()
aux_ini

,month,sales
0,septiembre 2021,32192485.00


In [6]:
import re
nielsen = pd.read_csv(BASE_DIR + f'nielsen_{CPG}.csv',
                      sep='\t',encoding='utf-16'
                     ).drop_duplicates(subset=['UPC','PRODUCT ID'])
display(nielsen['UPC'].head())
nielsen['UPC'] = nielsen['UPC'].apply(lambda x: re.sub('[^\d]','',str(x)))
nielsen['UPC'] = nielsen['UPC'].astype(str).str.strip().str.split().str[0].fillna(0).astype(int)
display(nielsen['UPC'].head())
display(nielsen['UPC'].head())
nielsen.sample(2)

0    7501032908331
1    7501058767226
2      19200858397
3    7501032906337
4    7501032924003
Name: UPC, dtype: object

0    7501032908331
1    7501058767226
2      19200858397
3    7501032906337
4    7501032924003
Name: UPC, dtype: int64

0    7501032908331
1    7501058767226
2      19200858397
3    7501032906337
4    7501032924003
Name: UPC, dtype: int64

,UPC,FORMATO,PROVEEDOR GENERAL,PROVEEDOR ABIERTO,MARCA DETALLE,MARCA GENERICA,GRAMAJE,DESCRIPCION NIELSEN,PRODUCT ID,SEGMENTO,SUBSEGMENTO 1,SUBSEGMENTO 2,SUBSEGMENTO 3
1724,870748005106,NaN,OTHERS FABRICANTE UNIF.,OTHERS FABRICANTE UNIF.,OTHER MARCA UNIF.,OTHER MARCA UNIF.,NaN,SMELLS AROMA REED DIFUSER VANILLA VELVET FRSCO...,NaN,AIR CARE,AIRFRESHENERS,LIQUIDS BASIC,NaN
3519,7501032909826,NaN,SC JOHNSON,SC JOHNSON,GLADE,GLADE,NaN,GLADE SENSATIONS HAWAIIAN BREEZE APARATO 8GR ...,NaN,AIR CARE,AIRFRESHENERS,OTROS(RMN),NaN


## Functions

In [7]:
import time
import numpy as np
from IPython.lib.display import Audio

start = time.time()
def time_exp(x):
    minutes, seconds = np.floor(x/60), 60*(x/60-np.floor(x/60))
    print(f"{'{:.0f}'.format(minutes)} minutos con {'{:.2f}'.format(seconds)} segundos")
    
def tono(a = 1000, b = 700, play_time_seconds = 1, framerate = 4410):
    t = np.linspace(0, play_time_seconds, framerate*play_time_seconds)*np.pi
    return Audio(np.sin(a*t) + np.sin(b*t), rate = framerate, autoplay=True)

In [8]:
def limpio(data,month_col,to_drop):
    df = data.copy()
    df[['mes','anio']] = df[month_col].str.split(expand=True)
    meses = dict(zip(['enero','febrero','marzo','abril','mayo','junio','julio',
                      'agosto','septiembre','octubre','noviembre','diciembre'],range(1,13)))
    df['mes'] = df['mes'].replace(meses)
    df.drop(columns=to_drop,inplace=True)
    df.columns = [str(x).replace('.','').replace(' ','_').lower() for x in df.columns]
    for col in ['brand_filter','cpg_filter']:
        df[f'{col[:-7]}_id'] = df[col].apply(lambda x: ','.join(re.findall('\s\((\d+)\)',str(x))))
        df[f'{col[:-7]}_name'] = df[col].apply(lambda x: re.sub('\s\(\d+\)','',str(x)))
    return df

## Transform

### Barcodes

In [9]:
df['barcodes'] = df['barcodes'].str.replace('{','').str.replace('}','')
df['barcodes'] = df['barcodes'].str.split(',',n=1).fillna('.').apply(lambda x:x[0])
df['barcodes'].sample(4)

/var/folders/j4/1p33_bh96yn8pdz3b_5t80hm0000gn/T/ipykernel_1430/2845833164.py:1: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will*not* be treated as literal strings when regex=True.
  df['barcodes'] = df['barcodes'].str.replace('{','').str.replace('}','')


20319    7501022150801
7096     7501025400842
11913    4973513134639
23267    7706800227007
Name: barcodes, dtype: object

### Merge

In [10]:
df = df.astype(str).merge(nielsen.astype(str).rename(columns={'UPC':'barcodes',
                                                              'PRODUCT ID':'product id'}),
                          on=['barcodes'] if nielsen_cat else ['barcodes','product id'],
                          how='left' if nielsen_cat else 'left')
if nielsen_cat:
    df = df.rename(columns={'product id_x':'product_id'}).drop(columns='product id_y')

df.sample(2)

,month,country,city name,store_name,category en,category_id,product id,product name,barcodes,package,...,PROVEEDOR GENERAL,PROVEEDOR ABIERTO,MARCA DETALLE,MARCA GENERICA,GRAMAJE,DESCRIPCION NIELSEN,SEGMENTO,SUBSEGMENTO 1,SUBSEGMENTO 2,SUBSEGMENTO 3
11468,septiembre 2021,MX,Ciudad de México,La Comer,Bathroom and kitchen cleaning,988,121465,Deshumificador ambiental aroma primavera 400 g,7501454344007,nan,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10660,septiembre 2021,MX,Ciudad de México,Fresko,Laundry Detergent,984,1106701,Quitamanchas crystal white sin cloro,7501058715890,1.8 kg,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### CPG

In [11]:
aux_fin = df.copy()
aux_fin['sales'] = aux_fin['sales'].astype(str).str.replace(',','').astype(float)
aux_fin = aux_fin.groupby('month  ')['sales'].sum().reset_index()
aux = aux_ini.copy()
aux = aux.set_index('month  ').join(aux_fin.set_index('month  '),rsuffix='_fin')
aux['dif'] = aux['sales_fin'] - aux['sales']
aux['dif_p'] = aux['sales_fin'] / (aux['sales']+1e-10) - 1
aux

,sales,sales_fin,dif,dif_p
month,,,,
septiembre 2021,32192485.00,32192485.00,0.00,0.00


In [12]:
prod_cpg = pd.read_csv(os.path.join(BASE_DIR,'pbbc.csv'),
                       #sep='\t',encoding='utf-16'
                      )
prod_cpg.sample(4)

,n,product_id,barcodes,category_id,category_name,brand_id,brand_name,cpg_id,cpg_name
116208,1,3838767,41457161326,1498,Accesorios para cabello,466,Goody,349,Biologic Group OGX
28886,1,212909,7501056359119,978,Accesorios de belleza,878,Pond's,44,Unilever
39366,1,313287,22000018489,750,Dulces,509,Skittles,53,Mars MX Candy
13866,1,143670,7501299307526,107,"Músculos, articulaciones y alivio del dolor",8561,Liomont,1110,Liomont


In [13]:
prod_cpg.drop_duplicates(('barcodes' if nielsen_cat else 'product_id'),inplace=True)
prod_cpg = prod_cpg.astype(str).rename(columns={'product_id':'product id'})
prod_cpg['cpg filter'] = prod_cpg['cpg_name']+" ("+prod_cpg['cpg_id'].astype(str)+")"
prod_cpg.sample(4)

,n,product id,barcodes,category_id,category_name,brand_id,brand_name,cpg_id,cpg_name,cpg filter
34838,1,275078,8410010272526,1338,Aceites de oliva,796,Carbonell,394,Deoleo,Deoleo (394)
48555,1,337914,7500435019880,1496,Limpieza y cuidado del cabello,273,Head & Shoulders,1,P&G MX,P&G MX (1)
1256,1,93047,7501017363735,974,Cuidado femenino,538,Kotex,50,Kimberly Clark,Kimberly Clark (50)
97901,1,2177503,71691532026,1539,Basureros,3078,Rubbermaid,975,Newell Brands,Newell Brands (975)


In [14]:
df = df.merge(prod_cpg[[('barcodes' if nielsen_cat else 'product id'),'cpg filter']],
              on=('barcodes' if nielsen_cat else 'product id'),how='left')

In [15]:
aux_fin = df.copy()
aux_fin['sales'] = aux_fin['sales'].astype(str).str.replace(',','').astype(float)
aux_fin = aux_fin.groupby('month  ')['sales'].sum().reset_index()
aux = aux_ini.copy()
aux = aux.set_index('month  ').join(aux_fin.set_index('month  '),rsuffix='_fin')
aux['dif'] = aux['sales_fin'] - aux['sales']
aux['dif_p'] = aux['sales_fin'] / (aux['sales']+1e-10) - 1
aux

,sales,sales_fin,dif,dif_p
month,,,,
septiembre 2021,32192485.00,32192485.00,0.00,0.00


### Limpio

In [16]:
df = limpio(df,'month  ',
            ['month  ','country ','avg ticket','found rate','fulfillment','frequency'])
df.sample(2)

,city_name,store_name,category_en,category_id,product_id,product_name,barcodes,package,product_sku,brand_filter,...,subsegmento_1,subsegmento_2,subsegmento_3,cpg_filter,mes,anio,brand_id,brand_name,cpg_id,cpg_name
36684,Querétaro,La Comer,Body lotion & soaps,964,324710,Repuesto jabón líquido manos aroma kiwi,7506267907999,1050 ml,nan,Blumen (3990),...,NaN,NaN,NaN,NaN,9,2021,3990,Blumen,,nan
21027,Hermosillo,Soriana,Bathroom and kitchen cleaning,988,315408,Cloro aroma a lavanda,7501025402242,3.78 l,322224,Blancatel (6545),...,NaN,NaN,NaN,Alen (20),9,2021,6545,Blancatel,20,Alen


In [17]:
aux_fin = df.copy()
aux_fin['sales'] = aux_fin['sales'].astype(str).str.replace(',','').astype(float)
aux_fin = aux_fin.groupby('mes')['sales'].sum().reset_index()
aux_fin['mes'] = aux_fin['mes'].astype(int).map(dict(zip(range(1,13),
                                               ['ene','feb','mar','abr','may','jun',
                                                'jul','ago','sep','oct','nov','dic'])))
aux = aux_ini.copy()
aux['month  '] = aux['month  '].str.split().str[0].str[:3]
aux = aux.set_index('month  ').join(aux_fin.set_index('mes'),rsuffix='_fin')
aux['dif'] = aux['sales_fin'] - aux['sales']
aux['dif_p'] = aux['sales_fin'] / (aux['sales']+1e-10) - 1
aux

,sales,sales_fin,dif,dif_p
month,,,,
sep,32192485.00,32192485.00,0.00,0.00


In [18]:
aux = df[df['formato'].astype(str)=='nan'].copy()
df = df[~(df['formato'].astype(str)=='nan')].copy()
null_fill = {'proveedor_general':'cpg_name',
             'proveedor_abierto':'cpg_name',
             'marca_detalle':'brand_name',
             'marca_generica':'brand_name',
             'gramaje':'package',
             'descripcion_nielsen':'product_name',
             'segmento':'category_en'}
for x,y in null_fill.items():
    aux.loc[:,x] = aux.loc[:,y]
    
df = df.append(aux,ignore_index=True)

In [19]:
for col in df.columns:
    df[col] = df[col].astype(str).str.replace('nan','')
    
df = df.drop(columns='cpg_name').fillna('')
df.sample()

,city_name,store_name,category_en,category_id,product_id,product_name,barcodes,package,product_sku,brand_filter,...,segmento,subsegmento_1,subsegmento_2,subsegmento_3,cpg_filter,mes,anio,brand_id,brand_name,cpg_id
32890,Puebla,La Comer,Oral care,967,342252,Crema dental para la sensibilidad,7794640170133,Tubo 100 g,,Sensodyne (853),...,Oral care,,,,GSK MX (276),9,2021,853,Sensodyne,276


In [20]:
brand_cols = ['formato','proveedor_general','proveedor_abierto','marca_detalle','marca_generica','gramaje','descripcion_nielsen','segmento','subsegmento_1','subsegmento_2','subsegmento_3']
display(df[brand_cols].sample())

for col in brand_cols:
    df[col] = df[col].str.strip().str.title()

display(df[brand_cols].sample())

,formato,proveedor_general,proveedor_abierto,marca_detalle,marca_generica,gramaje,descripcion_nielsen,segmento,subsegmento_1,subsegmento_2,subsegmento_3
39365,,Genomma Lab,Genomma Lab,Nordiko,Nordiko,100 g,Jabón desodorante Nórdico,Body lotion & soaps,,,


,formato,proveedor_general,proveedor_abierto,marca_detalle,marca_generica,gramaje,descripcion_nielsen,segmento,subsegmento_1,subsegmento_2,subsegmento_3
30195,,Colgate Palmolive,Colgate Palmolive,Colgate,Colgate,75 Ml,Pasta Dental Liga De La Justicia,Oral Care,,,


In [21]:
aux_fin = df.copy()
aux_fin['sales'] = aux_fin['sales'].astype(str).str.replace(',','').astype(float)
aux_fin = aux_fin.groupby('mes')['sales'].sum().reset_index()
aux_fin['mes'] = aux_fin['mes'].astype(int).map(dict(zip(range(1,13),
                                               ['ene','feb','mar','abr','may','jun',
                                                'jul','ago','sep','oct','nov','dic'])))
aux_fin

,mes,sales
0,sep,32192485.00


### Marca genérica

In [22]:
df.loc[df[df['marca_generica']==''].index, 'marca_generica'] = df.loc[df[df['marca_generica']==''].index, 'marca_detalle']
df[df['marca_generica']=='']

,city_name,store_name,category_en,category_id,product_id,product_name,barcodes,package,product_sku,brand_filter,...,segmento,subsegmento_1,subsegmento_2,subsegmento_3,cpg_filter,mes,anio,brand_id,brand_name,cpg_id


## Export

### Same product_id as past

In [23]:
print(len(df))
if past:
    som = pd.ExcelFile(os.path.join('/'.join(BASE_DIR.split('/')[:-2]),
                                    f'SoM_{SOM_CPG}.xlsx')).parse(sheet_name=0)
    df = df.merge(som.drop_duplicates('product_id')[['product_id']].reset_index(drop=True).astype(str))
    
print(len(df))

41893
34734


In [24]:
aux_fin = df.copy()
aux_fin['sales'] = aux_fin['sales'].astype(str).str.replace(',','').astype(float)
aux_fin = aux_fin.groupby('mes')['sales'].sum().reset_index()
aux_fin['mes'] = aux_fin['mes'].astype(int).map(dict(zip(range(1,13),
                                               ['ene','feb','mar','abr','may','jun',
                                                'jul','ago','sep','oct','nov','dic'])))

In [25]:
aux = aux_ini.copy()
aux['month  '] = aux['month  '].str.split().str[0].str[:3]
aux = aux.set_index('month  ').join(aux_fin.set_index('mes'),rsuffix='_fin')
aux['dif'] = aux['sales_fin'] - aux['sales']
aux['dif_p'] = aux['sales_fin'] / (aux['sales']+1e-10) - 1

### Ordered columns

In [26]:
export = df[['city_name', 'store_name', 'category_en', 'category_id', 'product_id',
             'product_name', 'barcodes', 'package', 'product_sku', 'brand_name',
             'sales', 'orders', 'users', 'units_found', 'units_replaced',
             'units_requested', 'formato', 'proveedor_general', 'proveedor_abierto',
             'marca_detalle', 'marca_generica', 'gramaje', 'descripcion_nielsen',
             'segmento', 'subsegmento_1', 'subsegmento_2', 'subsegmento_3',
             'mes', 'anio']].copy()
export.sort_values(['anio','mes'],inplace=True)
export.sample(4)

,city_name,store_name,category_en,category_id,product_id,product_name,barcodes,package,product_sku,brand_name,...,marca_detalle,marca_generica,gramaje,descripcion_nielsen,segmento,subsegmento_1,subsegmento_2,subsegmento_3,mes,anio
10268,Ciudad de México,Soriana,Air Fresheners & Candles,990,355837,Aromatizante en aerosol magnolia y cherry,7501058711670,226 g,,Air Wick,...,Air Wick,Air Wick,226 G,Aromatizante En Aerosol Magnolia Y Cherry,Air Fresheners & Candles,,,,9,2021
20612,Querétaro,La Comer,Laundry Detergent,984,1153392,Detergente líquido concentrado,7500435139908,1.2 l,7500435139908,Ariel,...,Ariel,Ariel,1.2 L,Detergente Líquido Concentrado,Laundry Detergent,,,,9,2021
27280,Ciudad de México,Chedraui,Air Fresheners & Candles,990,3480859,Vela taco rústico rojo granat,8436551846422,1 pieza,000000000003601132,Ambientair,...,Ambientair,Ambientair,1 Pieza,Vela Taco Rústico Rojo Granat,Air Fresheners & Candles,,,,9,2021
33022,Guadalajara,Soriana,Floor and furniture cleaning,989,2265072,Limpiador multiusos limón,7500093751351,5 l,,Precissimo,...,Precissimo,Precissimo,5 L,Limpiador Multiusos Limón,Floor And Furniture Cleaning,,,,9,2021


In [27]:
export.to_csv(os.path.join(BASE_DIR,f'som_{CPG}.csv'),index=False,sep='\t',encoding='utf-16')

## Fin

In [28]:
aux

,sales,sales_fin,dif,dif_p
month,,,,
sep,32192485.00,28774839.00,-3417646.00,-0.11


In [29]:
time_exp(time.time() - start)
tono()

2 minutos con 8.58 segundos
